In [120]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Data loading
df=pd.read_csv('Flipkart_DATA_Mobiles_692.csv')
df

,Title,MRP,Selling_Price,Specifications,Ratings,Discount,Review_Count
0,"POCO C75 5G (Enchanted Green, 64 GB) (4 GB RAM)","₹10,999","₹7,699",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB\n1...,4.3,30% off,"83,280 Ratings & 4,112 Reviews"
1,"POCO M6 Plus 5G (Graphite Black, 128 GB) (6 G...","₹15,999","₹10,080",6 GB RAM | 128 GB ROM | Expandable Upto 1 TB\n...,4.2,36% off,"59,597 Ratings & 3,573 Reviews"
2,"CMF by Nothing Phone 2 Pro (Black, 128 GB) (8...","₹22,999","₹18,999",8 GB RAM | 128 GB ROM | Expandable Upto 2 TB\n...,4.4,17% off,"15,881 Ratings & 1,311 Reviews"
3,"Motorola G85 5G (Viva Magenta, 128 GB) (8 GB ...","₹20,999","₹15,999",8 GB RAM | 128 GB ROM\n16.94 cm (6.67 inch) Fu...,4.4,23% off,"2,17,987 Ratings & 11,163 Reviews"
4,OPPO K13 5G with 7000mAh and 80W SUPERVOOC Cha...,"₹22,999","₹17,999",8 GB RAM | 128 GB ROM\n16.94 cm (6.67 inch) Di...,4.5,21% off,"24,615 Ratings & 1,677 Reviews"
...,...,...,...,...,...,...,...
687,"POCO C61 (Ethereal Blue, 64 GB) (4 GB RAM)","₹8,999","₹6,975",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB\n1...,4.1,22% off,"1,10,043 Ratings & 4,655 Reviews"
688,"Samsung M06 5G (Blazing Black, 128 GB) (4 GB ...","₹13,999","₹10,390",4 GB RAM | 128 GB ROM\n17.02 cm (6.7 inch) Dis...,3.9,25% off,270 Ratings & 30 Reviews
689,"REDMI Note 14 5G (Titan Black, 256 GB) (8 GB ...","₹24,999","₹19,817",8 GB RAM | 256 GB ROM\n16.94 cm (6.67 inch) Di...,4.2,20% off,"1,322 Ratings & 80 Reviews"
690,"REDMI Note 14 5G (Phantom Purple, 128 GB) (8 ...","₹22,999","₹17,319",8 GB RAM | 128 GB ROM\n16.94 cm (6.67 inch) Di...,4.2,24% off,"1,322 Ratings & 80 Reviews"


In [122]:
import re
# Extract brand and model
df['Brand'] = df['Title'].str.split().str[0]
df['Brand_Model'] = df['Title'].apply(lambda x: ' '.join(str(x).split()[:3]))

# Extract Color (text inside parentheses before first comma)
df['Color'] = df['Title'].str.extract(r'\(([^,]+)')

# Clean currency symbols and commas from prices
df['MRP'] = df['MRP'].replace({'₹': '', ',': ''}, regex=True).astype(float)
df['Selling_Price'] = df['Selling_Price'].replace({'₹': '', ',': ''}, regex=True).astype(float)

# Extract Display Size (in inches)
df['Display_Size'] = df['Specifications'].str.extract(r'([\d.]+)\s*inch')

# Extract Battery Capacity
df['Battery'] = df['Specifications'].str.extract(r'(\d+)\s*mAh')

# Split specification lines for processor/camera
specs = df['Specifications'].astype(str).str.split('\n')

# Processor Extraction: grabs the first line containing 'Processor'
df['Processor'] = specs.apply(lambda x: next((s for s in x if 'Processor' in s), ''))

# Back Camera MP: Extract first MP before "Front" or "Selfie"
df['Back_Cam(MP)'] = df['Specifications'].str.extract(r'(\d+)\s*MP(?!.*?(Front|Selfie))', expand=False)[0]

# Front Camera MP: Look for Front or Selfie camera values
df['Front_Cam(MP)'] = df['Specifications'].str.extract(r'(\d+)\s*MP\s*(?:Front|Selfie)', expand=False)

# Discount: remove '% off'
df['Discount'] = df['Discount'].str.replace('% off', '', regex=False).astype(float)

# Extract Rating and Review Counts
df['Rating_Count'] = df['Review_Count'].str.extract(r'([\d,]+)\s*Ratings')[0]
df['Review_Count'] = df['Review_Count'].str.extract(r'([\d,]+)\s*Reviews')[0]

# Remove commas and convert to numeric
df['Rating_Count'] = df['Rating_Count'].str.replace(',', '', regex=False).astype('Int64')
df['Review_Count'] = df['Review_Count'].str.replace(',', '', regex=False).astype('Int64')

# Add Platform
df['Platform'] = 'Flipkart'

In [124]:
# Ensure Specifications is string
df['Specifications'] = df['Specifications'].astype(str)
# --- Extract RAM in GB only (preceded by 'RAM') ---
df['RAM'] = df['Specifications'].str.extract(r'(?i)(\d+)\s*GB\s*RAM')[0]
df['RAM'] = pd.to_numeric(df['RAM'], errors='coerce').astype('Int64')

import re
def extract_rom_gb_only(text):
    if isinstance(text, str):
        # Find only values labeled GB (skip MB)
        matches = re.findall(r'(\d+)\s*GB', text)
        if matches:
            return max(map(int, matches))  # ROM = largest GB value
    return None
# Apply the function
df['ROM'] = df['Specifications'].apply(extract_rom_gb_only)

In [128]:
df

,Title,MRP,Selling_Price,Specifications,Ratings,Discount,Review_Count,Brand,Brand_Model,Color,Display_Size,Battery,Processor,Back_Cam(MP),Front_Cam(MP),Rating_Count,Platform,RAM,ROM
0,"POCO C75 5G (Enchanted Green, 64 GB) (4 GB RAM)",10999.0,7699.0,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB\n1...,4.3,30.0,4112,POCO,POCO C75 5G,Enchanted Green,6.88,5160,4s Gen 2 5G Processor,NaN,5,83280,Flipkart,4,64.0
1,"POCO M6 Plus 5G (Graphite Black, 128 GB) (6 G...",15999.0,10080.0,6 GB RAM | 128 GB ROM | Expandable Upto 1 TB\n...,4.2,36.0,3573,POCO,POCO M6 Plus,Graphite Black,6.79,5030,Snapdragon 4 Gen2 AE Processor,NaN,13,59597,Flipkart,6,128.0
2,"CMF by Nothing Phone 2 Pro (Black, 128 GB) (8...",22999.0,18999.0,8 GB RAM | 128 GB ROM | Expandable Upto 2 TB\n...,4.4,17.0,1311,CMF,CMF by Nothing,Black,6.77,5000,Dimensity 7300 Pro 5G Processor,NaN,16,15881,Flipkart,8,128.0
3,"Motorola G85 5G (Viva Magenta, 128 GB) (8 GB ...",20999.0,15999.0,8 GB RAM | 128 GB ROM\n16.94 cm (6.67 inch) Fu...,4.4,23.0,11163,Motorola,Motorola G85 5G,Viva Magenta,6.67,5000,6s Gen 3 Processor,NaN,32,217987,Flipkart,8,128.0
4,OPPO K13 5G with 7000mAh and 80W SUPERVOOC Cha...,22999.0,17999.0,8 GB RAM | 128 GB ROM\n16.94 cm (6.67 inch) Di...,4.5,21.0,1677,OPPO,OPPO K13 5G,Prism Black,6.67,7000,Snapdragon 6 Gen 4 Processor,NaN,16,24615,Flipkart,8,128.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,"POCO C61 (Ethereal Blue, 64 GB) (4 GB RAM)",8999.0,6975.0,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB\n1...,4.1,22.0,4655,POCO,POCO C61 (Ethereal,Ethereal Blue,6.71,5000,Helio G36 Processor,NaN,5,110043,Flipkart,4,64.0
688,"Samsung M06 5G (Blazing Black, 128 GB) (4 GB ...",13999.0,10390.0,4 GB RAM | 128 GB ROM\n17.02 cm (6.7 inch) Dis...,3.9,25.0,30,Samsung,Samsung M06 5G,Blazing Black,6.7,5000,,50,NaN,270,Flipkart,4,128.0
689,"REDMI Note 14 5G (Titan Black, 256 GB) (8 GB ...",24999.0,19817.0,8 GB RAM | 256 GB ROM\n16.94 cm (6.67 inch) Di...,4.2,20.0,80,REDMI,REDMI Note 14,Titan Black,6.67,5110,,50,NaN,1322,Flipkart,8,256.0
690,"REDMI Note 14 5G (Phantom Purple, 128 GB) (8 ...",22999.0,17319.0,8 GB RAM | 128 GB ROM\n16.94 cm (6.67 inch) Di...,4.2,24.0,80,REDMI,REDMI Note 14,Phantom Purple,6.67,5110,,50,NaN,1322,Flipkart,8,128.0


In [134]:
df['ROM'].unique()

array([  64.,  128.,   nan,  512.,  256.,   32., 1500.])

In [132]:
# Set a lower bound to filter out MB values (commonly 16 or 1500 MB)
df['ROM'] = df['ROM'].apply(lambda x: x if pd.notna(x) and x >= 32 else None)


In [138]:
# Calculate Discount Price
df['Discount_Price'] = df['MRP'] - df['Selling_Price']
# Convert Discount_Price to float, forcing errors to NaN
df['Discount_Price'] = pd.to_numeric(df['Discount_Price'], errors='coerce')

In [136]:
# Convert numeric columns to proper data types
int_columns = ['Review_Count', 'Rating_Count', 'RAM', 'ROM', 'Battery', 'Front_Cam(MP)', 'Back_Cam(MP)']
float_columns = ['MRP', 'Selling_Price','Ratings', 'Discount', 'Display_Size'] #'Discount_Price'

for col in int_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # Nullable integer type
for col in float_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)

In [ ]:
# this data having some negative values then i remove that

In [140]:
df = df[df['Discount_Price'] >= 0]  # Keep only positive discounts

In [142]:
df= df[['Brand', 'Brand_Model', 'Color', 'Platform', 'MRP', 'Selling_Price', 'Discount_Price',
    'Discount', 'RAM', 'ROM', 'Display_Size', 'Battery', 
    'Front_Cam(MP)', 'Back_Cam(MP)', 'Processor','Ratings', 'Rating_Count', 'Review_Count'
    ]]

#### Data Validation

#### Checking column wise data

In [147]:
df[df['Selling_Price'] > df['MRP']]

,Brand,Brand_Model,Color,Platform,MRP,Selling_Price,Discount_Price,Discount,RAM,ROM,Display_Size,Battery,Front_Cam(MP),Back_Cam(MP),Processor,Ratings,Rating_Count,Review_Count


In [26]:
import numpy as np
df.loc[df['Discount_Price'] < 0, 'Discount_Price'] = np.nan  # or 0

In [149]:
df.columns

Index(['Brand', 'Brand_Model', 'Color', 'Platform', 'MRP', 'Selling_Price',
       'Discount_Price', 'Discount', 'RAM', 'ROM', 'Display_Size', 'Battery',
       'Front_Cam(MP)', 'Back_Cam(MP)', 'Processor', 'Ratings', 'Rating_Count',
       'Review_Count'],
      dtype='object')

In [128]:
# Checking every column data

In [130]:
df['Brand_Model'].unique()

array(['POCO C75 5G', 'POCO M6 Plus', 'CMF by Nothing', 'Motorola G85 5G',
       'OPPO K13 5G', 'Nokia 105 Single', 'MOTOROLA g35 5G', 'POCO F7 5G',
       'Motorola g45 5G', 'vivo T4 Lite', 'Samsung Guru Music',
       'Samsung Guru 1200', 'Samsung Galaxy F16', 'REDMI A5 (Just',
       'MOTOROLA Edge 60', 'Apple iPhone 16', 'POCO M7 5G',
       'Infinix Note 50s', 'OPPO K13x 5G', 'vivo T4x 5G', 'REDMI 13 5G',
       'POCO C71 (Power', 'itel it5330 |', 'POCO M7 Pro',
       'Motorola Edge 50', 'Tecno Spark Go', 'Samsung SM 310E',
       'OnePlus Nord CE4', 'POCO X7 5G', 'realme P3x 5G', 'IQOO Z10X 5G',
       'Samsung Galaxy A35', 'realme 14x 5G', 'Samsung Guru1200 (Black)',
       'Infinix Note 50x', 'Nokia 105 Classic',
       'Kechaoda A26_black (Black)', 'Samsung M05 (Mint',
       'Samsung Galaxy M35', 'Tecno Pova 6', 'vivo T4 5G', 'realme P3 5G',
       'itel Keypad Mobile', 'realme GT 6', 'realme P2 Pro',
       'realme C61 (Marble', 'Nothing Phone (3a)', 'vivo Y19e (Titanium',

In [151]:
# this column having some symbols 
df['Brand_Model'] = df['Brand_Model'].str.replace(r"[\(\'\[\|].*", '', regex=True).str.strip()

In [153]:
# colur column has some unwanted data we can replce that
df['Color'] = (df['Color'].str.replace(r'[(),|]', '', regex=True)
    .str.replace(r"\b(5G|RAM|songs|Dual SIM|Inch|Display|Pro|8MB|2025|2'|Big|Long-Lasting|AI|Battery|CM181|with|1500|pre-loaded|2a)\b", '', regex=True)
    .str.replace(r'\s+', ' ', regex=True).str.strip())

In [155]:
df['Processor'] = (df['Processor'].str.replace(r'[(),|®™,+]', '', regex=True)
    .str.replace(r'up to(?: \d+\.?\d* GHz x \d+)+', '', regex=True)
    .str.replace(r'Max clock speed: \d+ x \w+@\d+\.?\d*GHz \d+ x \w+@\d+\.?\d*GHz', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True).str.strip())

In [157]:
# After Extraction and data validation
df

,Brand,Brand_Model,Color,Platform,MRP,Selling_Price,Discount_Price,Discount,RAM,ROM,Display_Size,Battery,Front_Cam(MP),Back_Cam(MP),Processor,Ratings,Rating_Count,Review_Count
0,POCO,POCO C75 5G,Enchanted Green,Flipkart,10999.0,7699.0,3300.0,30.0,4,64,6.88,5160,5,<NA>,4s Gen 2 5G Processor,4.3,83280,4112
1,POCO,POCO M6 Plus,Graphite Black,Flipkart,15999.0,10080.0,5919.0,36.0,6,128,6.79,5030,13,<NA>,Snapdragon 4 Gen2 AE Processor,4.2,59597,3573
2,CMF,CMF by Nothing,Black,Flipkart,22999.0,18999.0,4000.0,17.0,8,128,6.77,5000,16,<NA>,Dimensity 7300 Pro 5G Processor,4.4,15881,1311
3,Motorola,Motorola G85 5G,Viva Magenta,Flipkart,20999.0,15999.0,5000.0,23.0,8,128,6.67,5000,32,<NA>,6s Gen 3 Processor,4.4,217987,11163
4,OPPO,OPPO K13 5G,Prism Black,Flipkart,22999.0,17999.0,5000.0,21.0,8,128,6.67,7000,16,<NA>,Snapdragon 6 Gen 4 Processor,4.5,24615,1677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,POCO,POCO C61,Ethereal Blue,Flipkart,8999.0,6975.0,2024.0,22.0,4,64,6.71,5000,5,<NA>,Helio G36 Processor,4.1,110043,4655
688,Samsung,Samsung M06 5G,Blazing Black,Flipkart,13999.0,10390.0,3609.0,25.0,4,128,6.70,5000,<NA>,50,,3.9,270,30
689,REDMI,REDMI Note 14,Titan Black,Flipkart,24999.0,19817.0,5182.0,20.0,8,256,6.67,5110,<NA>,50,,4.2,1322,80
690,REDMI,REDMI Note 14,Phantom Purple,Flipkart,22999.0,17319.0,5680.0,24.0,8,128,6.67,5110,<NA>,50,,4.2,1322,80


In [163]:
# saving cleaned data
df.to_csv("Flipkart_Final.csv", index=False)

#### deoping nan values rows becasues those are keypad mobiles

In [165]:
df=pd.read_csv('Flipkart_Final.csv')

In [169]:
df1=df.dropna()

In [173]:
df1.to_csv('Flipkart_Final.csv',index=False)

### Mergeing to Data sets in one data set (Flipkart and Amazon)

In [178]:
# Cheking to data sets having same columns or not
df1=pd.read_csv('Flipkart_Final.csv')
df2=pd.read_csv('Amazon_Final.csv')

df2.columns = df1.columns

In [180]:
import pandas as pd

# Load both CSV files
df1=pd.read_csv('Flipkart_Final.csv')
df2=pd.read_csv('Amazon_Final.csv')

# Combine them vertically (file1 on top, file2 below)
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined data
combined_df.to_csv("Ecommerce_Final.csv", index=False)